In [1]:
import pandas as pd
import collections
import json
import csv
import numpy as np

# Read csv to dataframe(ANSI encoding)

## Column DataType
- name: str
- owners:int
- developer: str
- positive:int
- publisher:str
- price:float
- initialprice:float

In [ ]:
file = pd.read_csv('GameData.csv' ,low_memory=0, header=0 , keep_default_na=False , encoding='ANSI')

# Remove unrealated Data

- accounting
- animation & modeling
- game developlemnt
- video production
- photo editing
- web-publisting
- utilities
- autdio production
- software training

In [ ]:
#Removing data from Genre   
count = 0
tempList = [] #new tempList
for entry in file['genre'].values.tolist(): 
    
    if("Accounting" in entry or 'Animation & Modeling' in entry or 'Game Development' in entry or 'Video Production' in entry or 'Animation & Modeling' in entry or 'Photo Editing' in entry or 'Web Publishing' in entry or 'Utilities' in entry or 'Audio Production' in entry or 'Software Training' in entry): 
        tempList.append(count)#append 
    count += 1 #inc
    
file = file.drop(tempList)

# Merging similar columns

- Action  & adventure -- Act_Adv
- violent & gore & sexual content & nudity -- 18+

In [ ]:
tempList = [] #new tempList
for entry in file['genre'].values.tolist(): 
    if("Violent" in entry or "Gore" in entry or "Sexual Content" in entry or "Nudity" in entry):
        tempList.append(1)
    else:
        tempList.append(0)
        
#Append column
file['18+'] = tempList

tempList = [] #new tempList
for entry in file['genre'].values.tolist(): 
    if("Action" in entry or "Adventure" in entry):
        tempList.append(1)
    else:
        tempList.append(0)
        
#Append column
file['Act_Adv'] = tempList

# Onehot genre column

- Casual
- Indie
- Free To Play
- RPG
- Strategy
- Sports
- Simulation
- Racing
- Massivley Multiplayer

In [ ]:
myList = ["Casual","Indie","Free To Play","RPG","Strategy","Sports","Simulation","Racing","Massivley Multiplayer"]

for genre in myList:
    tempList = [] #new tempList
    
    #Loop through all genre in list
    for entry in file['genre'].values.tolist(): 
        #If genre found set val to 1 else 0
        if(genre in entry):
            tempList.append(1)
        else:
            tempList.append(0)
    #Append column
    file[genre] = tempList

# Onehot language column

- English
- Chinese
- French
- German
- Italian
- Spanish

## And extra columns with total language supported

## Function to set value for onehot column

In [ ]:
def languagefilter(lst, check):
    newlist = []
    print('\nList: ', lst, '\n')
    length = len(lst)
    for i in range(length):
        if "English" in lst[i]:
            check[0] = "1"
            lst[i] = "removed"
        elif "Chinese" in lst[i]:
            check[1] = "1"
            lst[i] = "removed"
        elif "French" in lst[i]:
            check[2] = "1"
            lst[i] = "removed"
        elif "German" in lst[i]:
            check[3] = "1"
            lst[i] = "removed"
        elif "Italian" in lst[i]:
            check[4] = "1"
            lst[i] = "removed"
        elif "Spanish" in lst[i]:
            check[5] = "1"
            lst[i] = "removed"
        
    for j in lst:
        print("j: ", j)
        if j == "removed":
            continue
        else:
            newlist.append(j)
    
    
    if check[5] == "1":
        newlist.insert(0, "Spanish")
    
    if check[4] == "1":
        newlist.insert(0, "Italian")
    
    if check[3] == "1":
        newlist.insert(0, "German")
    
    if check[2] == "1":
        newlist.insert(0, "French")
   
    if check[1] == "1":
        newlist.insert(0, "Chinese")
    
    if check[0] == "1":
        newlist.insert(0, "English")
    
    check[6] = len(newlist)
    check[7] = newlist
    print(check, '\n\n')
    return check

In [ ]:
#Get lanaguage column
lang = pd.DataFrame(file, columns=['languages'])

#Top used language
newlang = pd.DataFrame(columns=["English", "Chinese", "French", "German", "Italian", "Spanish", "Total Languages", "Collection"])

#New List
langlist = []

#Loop throguh all columns
for i in range(len(lang)):
    
    #New columns to append
    toappend = ["0", "0", "0", "0", "0", "0", 0, lang['languages'][i]]
    
    #If no language found list = empty list else split language to list
    if lang['languages'][i] == '':
        strlist = []
    else:
        string = lang['languages'][i]
        strlist = string.split(', ')
    
    #Check thorught list and set append column
    toappend = languagefilter(strlist, toappend)
    
    #append column
    strseries = pd.Series(toappend, index=newlang.columns)
    newlang = newlang.append(strseries, ignore_index=True)

#concat dataframe
file = pd.concat([file, newlang], axis=1)

# Calculating review Score Column 

## review_score = (positive score / positive score * negativescore) * 100

In [ ]:
data = file[['positive', 'negative']]

pos, neg, total = 0, 0, 0
for i in data['positive']:
    if i == 0:
        pos += 1
        
for j in data['negative']:
    if j == 0:
        neg += 1
        
for i in range(0,len(data)):
    if data['positive'][i] == 0 and data['negative'][i] == 0:
        data['review'][i] = 0
    else:
        data['review'][i] = int((data['positive'][i] / (data['positive'][i] + data['negative'][i])) * 100)


# Convert price to categorical data

## Create a dictonary with 
- Key: Amount of owner
- Value: Number maps to owner

### Then maps owner to category

### Categorical owners
- 0-20000: 0
- 20001-50000: 1
- 50001-100000: 2
- 100001-200000: 3
- 200001-500000: 4
- 500001-1000000: 5
- 1000001-2000000: 6 
- 2000001-5000000: 7 
- 5000001-10000000: 8 
- 10000001-50000000: 9 
- 50000001-20000000: 10
- 20000001-100000000: 11 
- 100000001-200000000: 12

In [2]:
#Get owners row
owners = pd.DataFrame(file['owners'])

#Make val to list
tempList = owners.values.tolist()

#New Dic
tempDic = {}

#Counter
count = 0

#Create Dic 
for data in tempList:
    if(tempDic.get(str(data)) == None):
        tempDic[str(data)] = count
        count += 1       
        
tempList = []
        
#Maps owner to category
for row in owners.values.tolist():
    tempList.append(tempDic[str(row)])

#Append column
file["owner_cat"] = tempList

{'[20000]': 0, '[50000]': 1, '[100000]': 2, '[200000]': 3, '[500000]': 4, '[1000000]': 5, '[2000000]': 6, '[5000000]': 7, '[10000000]': 8, '[20000000]': 9, '[50000000]': 10, '[100000000]': 11, '[200000000]': 12}


# Generate new csv 

In [ ]:
file = file.to_csv('GameData_cleaned.csv') #New CSV file